# Load and work with csv files
See [https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset](https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset)

In [17]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [21]:
B_all = pd.read_csv('/Users/flint/Data/yelp/csv/business.csv', header=0)

In [22]:
B_all.shape

(174567, 12)

In [23]:
B_all.head(2)

,id,name,postal,street,lat,lon,stars,review,is_open,city_id,city_name,state
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,85044,"4855 E Warner Rd, Ste B9",33.330690,-111.978599,4.0,22,True,D66C6A,Ahwatukee,AZ
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,15317,3101 Washington Rd,40.291685,-80.104900,3.0,11,True,60618F,McMurray,PA


In [24]:
C_all = pd.read_csv('/Users/flint/Data/yelp/csv/yelp_incat.csv')

In [25]:
C_all.shape

(668022, 2)

In [26]:
C_all.head()

,business,category
0,mmazCP1ZH0QsUqDS6OivFA,Seafood
1,Zh7k_33xMSPwm0UU7LsEIw,Education
2,EBa7-48XiJMmbbZ7VDhrjg,Shopping
3,IsUuk4HqkQaMPwpLGCHAtQ,Hair Extensions
4,WrBArrXMBdQHcdfudhneGw,Home Services


## Explore data with pandas and matplotlib

## Compute a ranking of business per category
See [https://en.wikipedia.org/wiki/Bayesian_average](https://en.wikipedia.org/wiki/Bayesian_average)

See also [datetime conversion](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

In [27]:
R_all = pd.read_csv('/Users/flint/Data/yelp/csv/review.csv')

In [29]:
R_all.shape

(307602, 4)

In [31]:
R_all.head()

,business,date,author,stars
0,ym5FOqL6tGbgynzwKMHZuw,2017-07-23,tKv1UeWDyEABNBrGhCfzUA,5
1,LCb2moBNkcBto_32rmZJ5A,2017-02-21,y2hYhjQECJG813jwNLnfKQ,3
2,ipFobdM6X_4SNMA9ZBfdAQ,2017-01-25,oZsYFsP7xBfcbO3CCmsNgQ,5
3,rnBm2hD9Y_RQ7LFo9S9AFA,2012-10-16,CWpVhVAzbkrHJK9_-KwIFQ,5
4,SAae44V24aD7y_HpgxjhPA,2017-06-10,DDMrvgE2Wz1IxStRtL5CYQ,3
